In [1]:
import torch
import gc

import asyncio
import threading


gc.collect()

torch.cuda.empty_cache()

C:\Users\thithilab\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import cv2
from sklearn import svm
import numpy as np
from datetime import datetime
import os
import time
from ultralytics.yolo.utils.files import increment_path
import torch
import gc
import pickle
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
import csv
#from pathlib import Path
import matplotlib.pyplot as plt
gc.collect()
torch.cuda.empty_cache()
import tensorflow as tf

from ultralytics.yolo.utils.plotting import Annotator

from ultralytics import YOLO

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#HONKAWA_DETECTOR = YOLO("D:/Python/SULarbmon/Python/env/yolov8_june/runs/segment/Honkawa_Weights_v1/weights/best.pt")  # load a custom model
#HOKKAIDO_DETECTOR = YOLO('runs/segment/train4/weights/best.pt')  # load a custom model

In [ ]:
SIZE_112=112

vgg_112 = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE_112,SIZE_112, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in vgg_112.layers:
	layer.trainable = False
    
#vgg.summary()  #Trainable parameters will be 0


In [3]:
SIZE_224 = 224

vgg_224 = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE_224, SIZE_224, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in vgg_224.layers:
	layer.trainable = False
    
#vgg.summary()  #Trainable parameters will be 0


In [ ]:
le_filename = '../HONKAWA_VGG_SVM_PROB_BLACKFILL/VGG_SVM_HONKAWA_112_blackfill.le'
#black_le_filename = '../HONKAWA_VGG_SVM_PROB_BLACKFILL/VGG_SVM_HONKAWA_112_blackfill.pkl'

predictor_filename ='../HONKAWA_VGG_SVM_PROB_BLACKFILL/VGG_SVM_HONKAWA_112_blackfill.pkl'
#black_predictor_filename ='../HONKAWA_VGG_SVM_PROB/VGG_SVM_HONKAWA_112_1103_3pfs_prob_BLACK.pkl'

predictor = pickle.load(open(predictor_filename, 'rb'))
lable_encoder = pickle.load(open(le_filename, 'rb'))


#black_predictor = pickle.load(open(black_predictor_filename, 'rb'))
#black_lable_encoder = pickle.load(open(black_le_filename, 'rb'))



In [4]:
Hokkaido_B_predictor_filename = '../PaperWeight_2024/HOKKAIDO_BLACK_SQUARE.pkl'
Hokkaido_B_le_filename = '../PaperWeight_2024/HOKKAIDO_BLACK_SQUARE.le'

Hokkaido_B_predictor = pickle.load(open(Hokkaido_B_predictor_filename,'rb'))
Hokkaido_B_lable_encoder = pickle.load(open(Hokkaido_B_le_filename,'rb'))

In [5]:
async def HOKKAIDO_B_SVM(image):  ## new with vgg
    
    vgg_timer = time.time()
    global N_TOP
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg_224.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    print('VGG took ',time.time()-vgg_timer)
    
    svm_timer = time.time()
    with tf.device('/GPU:0'):
        probabilities = Hokkaido_B_predictor.predict_proba(input_img_features)
    print('SVM took ',time.time()-svm_timer)
    top_classes_indices = np.argsort(probabilities[0])[::-1][:N_TOP]
    top_predicted_results = []
    top_probabilities = probabilities[0][top_classes_indices]
    
    
    svm_timer = time.time()
    for i in range(N_TOP):
        result = Hokkaido_B_lable_encoder.inverse_transform([top_classes_indices[i]])  #Reverse the label encoder to original name
        #print(i+1,' => ',result, ' is the top result with ',top_probabilities[i])
        top_predicted_results.append(int(result))
        top_predicted_results.append(str(top_probabilities[i]))
    #print(top_predicted_results)    
    print('LOOPING result took ',time.time()-svm_timer)
    return top_predicted_results
    
    return predicted_result
print("defined HOKKAIDO_B_SVM predictor")

defined HOKKAIDO_B_SVM predictor


In [6]:
Hokkaido_NB_predictor_filename = '../PaperWeight_2024/HOKKAIDO_NON_BLACK_SQUARE.pkl'
Hokkaido_NB_le_filename = '../PaperWeight_2024/HOKKAIDO_NON_BLACK_SQUARE.le'

Hokkaido_NB_predictor = pickle.load(open(Hokkaido_NB_predictor_filename,'rb'))
Hokkaido_NB_lable_encoder = pickle.load(open(Hokkaido_NB_le_filename,'rb'))

In [7]:

async def HOKKAIDO_NB_SVM(image):  ## new with vgg
    vgg_timer = time.time()
    global N_TOP
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg_224.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    print('VGG took ',time.time()-vgg_timer)
    
    svm_timer = time.time()
    with tf.device('/GPU:0'):
        probabilities = Hokkaido_NB_predictor.predict_proba(input_img_features)
    print('SVM took ',time.time()-svm_timer)
    top_classes_indices = np.argsort(probabilities[0])[::-1][:N_TOP]
    top_predicted_results = []
    top_probabilities = probabilities[0][top_classes_indices]
    
    
    svm_timer = time.time()
    for i in range(N_TOP):
        result = Hokkaido_NB_lable_encoder.inverse_transform([top_classes_indices[i]])  #Reverse the label encoder to original name
        #print(i+1,' => ',result, ' is the top result with ',top_probabilities[i])
        top_predicted_results.append(int(result))
        top_predicted_results.append(str(top_probabilities[i]))
    #print(top_predicted_results)    
    print('LOOPING result took ',time.time()-svm_timer)
    return top_predicted_results
    
    #return predicted_result
print("defined HOKKAIDO_NB_SVM predictor")

defined HOKKAIDO_NB_SVM predictor


In [ ]:
Sumiyoshi_predictor_filename = '../PaperWeight_2024/SUMIYOSHI.pkl'
Sumiyoshi_le_filename = '../PaperWeight_2024/SUMIYOSHI.le'

Sumiyoshi_predictor = pickle.load(open(Sumiyoshi_predictor_filename,'rb'))
Sumiyoshi_lable_encoder = picker.load(open(Sumiyoshi_le_filename,'rb'))

In [ ]:
def SUMIYOSHI_NB_SVM(image):  ## new with vgg
    
    global N_TOP
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg_224.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    probabilities = Sumiyoshi_predictor.predict_proba(input_img_features)
    
    top_classes_indices = np.argsort(probabilities[0])[::-1][:N_TOP]
    top_predicted_results = []
    top_probabilities = probabilities[0][top_classes_indices]
    for i in range(N_TOP):
        result = Sumiyoshi_lable_encoder.inverse_transform([top_classes_indices[i]])  #Reverse the label encoder to original name
        #print(i+1,' => ',result, ' is the top result with ',top_probabilities[i])
        top_predicted_results.append(int(result))
        top_predicted_results.append(str(top_probabilities[i]))
    #print(top_predicted_results)    
    return top_predicted_results
    
    return predicted_result
print("defined SUMIYOSHI_NB_SVM predictor")

In [ ]:
def HONKAWA_NB_SVM(image):  ## new with vgg
    global N_TOP
    global mapping_data
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    probabilities = predictor.predict_proba(input_img_features)
    
    top_classes_indices = np.argsort(probabilities[0])[::-1][:N_TOP]
    top_predicted_results = []
    top_probabilities = probabilities[0][top_classes_indices]
    for i in range(N_TOP):
        #result = lable_encoder.inverse_transform([top_classes_indices[i]])[0]  #Reverse the label encoder to original name
        #print(result)
        #if result in mapping_data:
        #    result = mapping_data[result] #to map incorrect order with correct order
    
        #print(i+1,' => ',result, ' is the top result with ',top_probabilities[i])
        #top_predicted_results.append(mapping_data[lable_encoder.inverse_transform([top_classes_indices[i]])[0]])
        top_predicted_results.append(lable_encoder.inverse_transform([top_classes_indices[i]])[0])
        top_predicted_results.append(str(top_probabilities[i]))
    #print(top_predicted_results)    
    return top_predicted_results

print("defined predictor")

In [ ]:
def Predict_SVM_BLACK(image):  ## new with vgg
    global N_TOP
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    probabilities = black_predictor.predict_proba(input_img_features)
    
    top_classes_indices = np.argsort(probabilities[0])[::-1][:N_TOP]
    top_predicted_results = []
    top_probabilities = probabilities[0][top_classes_indices]
    for i in range(N_TOP):
        result = black_lable_encoder.inverse_transform([top_classes_indices[i]])  #Reverse the label encoder to original name
        #print(i+1,' => ',result, ' is the top result with ',top_probabilities[i])
        top_predicted_results.append(int(result))
        top_predicted_results.append(str(top_probabilities[i]))
    #print(top_predicted_results)    
    return top_predicted_results

print("defined predictor")

In [8]:

###### load dataset and model


def check_withinROI_NEW(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    #if(x1<int(X1*(w/default)) or x2>int(X2*(w/default)) or y1<int(Y1_NEW*(h/default)) or y2>int(Y2_NEW*(h/default)) or x1>=int(X2*(w/default))):
    #    return False
    
    if(x1<150 or x2>1750):
        return False
    if(y2-y1<600 or x2-x1 <250): #1400 to 700 Before
        return False
    return True  

def check_withinROI_Resize(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    resize_x1=850#*(w/2992)
    resize_x2=1050#*(w/2992)
    resize_y1=Y1_NEW#Y1_NEW*(w/2992)
    resize_y2=Y1_NEW#Y2_NEW*(w/2992)
    #print(resize_x1, '  ',resize_y1, '  ',resize_x2, '  ',resize_y2)
    if(x1<int(resize_x1) or x2>int(resize_x2) or x1>=int(resize_x2)):
        return False
    if(y2 - y1>1400 or y2-y1<700): #1400 to 700 Before
        return False
    return True  


In [9]:
def HONKAWA_ROI(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    #if(x1<int(X1*(w/default)) or x2>int(X2*(w/default)) or y1<int(Y1_NEW*(h/default)) or y2>int(Y2_NEW*(h/default)) or x1>=int(X2*(w/default))):
    #    return False
    
    if(x1<150 or x2>1750):
        return False
    if(y2-y1<600 or x2-x1 <250): #1400 to 700 Before
        return False
    return True  

In [10]:
def HOKKAIDO_ROI(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    if(x1<int(X1*(w/default)) or x2>int(X2*(w/default)) or y1<int(Y1_NEW*(h/default)) or y2>int(Y2_NEW*(h/default)) or x1>=int(X2*(w/default))):
        return False
    if(y2 - y1>1350 or y2-y1<900): #1400 to 700 Before
        return False
    return True  

In [11]:
def Is_Duplicate_Id(y1,y2,id):
    global PREVIOUS_ID
    global PREVIOUS_Y1
    global PREVIOUS_Y2
    global PREVIOUS_LOCAL_IDS
    global CATTLE_LOCAL_ID
    
    try: 
        index = PREVIOUS_ID.index(id)
        #print('I reached here')
        if(PREVIOUS_Y1[index]+321<=y1 and PREVIOUS_Y2[index]+371<y2): #duplicate from bottom
         #   if(id in PREVIOUS_LOCAL_IDS):
         #   #print('id: ',id,' LOCAL_ID: ',CATTLE_LOCAL_ID)
         #       return PREVIOUS_LOCAL_IDS[id][0]
            
            #print('except')
            PREVIOUS_ID.append(CATTLE_LOCAL_ID)
            PREVIOUS_Y1.append(y1)
            PREVIOUS_Y2.append(y2)
            #print('New Cattle Id')
            CATTLE_LOCAL_ID+=1
            return CATTLE_LOCAL_ID
        #elif(PREVIOUS_Y[index]+400<center): #stepping back
        #    if(id in PREVIOUS_LOCAL_IDS):
        #        return PREVIOUS_LOCAL_IDS[id][0]
        else:
            #print('Oh. here ? really?')
            PREVIOUS_Y1[index]=y1 #duplicate is solved or no duplicate and just need for last y 
            PREVIOUS_Y2[index]=y2
            #return PREVIOUS_LOCAL_IDS[index][1]
            
            #update('PREVIOUS Y')
            return PREVIOUS_ID[index]
    except:
        #print(PREVIOUS_ID)
        #print(id)
        CATTLE_LOCAL_ID += 1
        #print('except')
        PREVIOUS_ID.append(CATTLE_LOCAL_ID)
        PREVIOUS_Y1.append(y1)
        PREVIOUS_Y2.append(y2)
        return id

In [ ]:

def HONKAWA_TRACKING(y,h,cow_srno=1):
    global STORED_IDS
    global STORED_MID_Y
    global STORED_MID_Y1
    global STORED_MID_Y2
    global STORED_MISS
    global LAST_SEEN_IDS
    global LAST_SEEN_ID_CENTROIDS
    global CATTLE_LOCAL_ID
    global IS_FIRST_CATTLE 
    y1 , y2 = y , y+h
    id = None
    
    if IS_FIRST_CATTLE:
        IS_FIRST_CATTLE = False
        id = CATTLE_LOCAL_ID
    #mid_y = y2
    mid_y = int(2*y + h)/2
    IS_NEW = True
    last_id = 999
    last_y1 = 0
    last_y2 = 0
    if(len(STORED_IDS)>0): 
        last_id = STORED_IDS[len(STORED_IDS)-1]
        last_y1 = STORED_MID_Y1[len(STORED_MID_Y1)-1]#max(STORED_MID_Y1)
        last_y2 = STORED_MID_Y2[len(STORED_MID_Y2)-1]#max(STORED_MID_Y2)
        MISSED_LEN = len(STORED_MISS)
        #if(IS_NEW):
        
        #    MISSED_LEN -=1
        removed = 0
        for i in range(MISSED_LEN):
            #print(i, ' missed index checking' )
            missed = STORED_MISS[i-removed]
            #print('checking ',i-removed, 'to remove')
            if(missed>20): #if missed 35 frames
    
                del STORED_MISS[i-removed]  
                del STORED_MID_Y[i-removed]
                del STORED_MID_Y1[i-removed]
                del STORED_MID_Y2[i-removed]
                del STORED_IDS[i-removed]
                removed+=1
                #print('removed')
                
    #clear misses
   
    
    threshold_1 = 130 #300
    threshold_2 = 130  #230
    Distance = 5
     
    #if mid_y <= 1300 or mid_y >= 700:
    #    threshold_1 = 320 #350
    #    threshold_2 = 370 #280
    for i in range(1,len(STORED_MID_Y)+1):
        #print(STORED_IDS[-i-1],STORED_MID_Y[-i-1],' ',i)
        
        
        #if(STORED_MID_Y[-i]+threshold_2>=mid_y and STORED_MID_Y[-i]-threshold_1<=mid_y): # and IS_NEW): #previous 150 #200
        if(STORED_MID_Y1[-i]-threshold_1<=y1 and STORED_MID_Y1[-i]+threshold_1>=y1) or (STORED_MID_Y2[-i]-threshold_2<=y2 and STORED_MID_Y2[-i]+threshold_2>=y2): # and IS_NEW): #previous 150 #200
            if(IS_NEW):
              
                
                Distance = abs(STORED_MID_Y1[-i] - y1)
                if(abs(STORED_MID_Y2[-i] - y2)<Distance):
                    Distance = abs(STORED_MID_Y2[-i] - y2)
                IS_NEW = False
                STORED_MID_Y1[-i] = y1
                STORED_MID_Y2[-i] = y2
                
                STORED_MISS[-i]=1
                id= STORED_IDS[-i]
                #print(Distance)
                #print(id)
                
            #try:
            #    exist_index = LAST_SEEN_IDS.index(id)
            #    if(LAST_SEEN_ID_CENTROIDS[exist_index]+200>y): # showing old id
            #        LAST_SEEN_ID_CENTROIDS[exist_index] = y
            #except:
            #print('corrected id :',STORED_IDS[-i])
            elif Distance >5:
                STORED_MISS[-i]+=1
            #else:
            #    STORED_MISS[-i]-=1 #reset count to 2 when not moving
        #elif(STORED_MID_Y1[-i]<=y1 and STORED_MID_Y2[-i]>=y2):
        #        STORED_MISS[-i]=30
        else:
            STORED_MISS[-i]+=1    
     
    if(IS_NEW == False):
        if(y1>last_y1+20 and y2>last_y2+20):
            IS_NEW = True
      
    if(IS_NEW):
        CATTLE_LOCAL_ID+=1
        id=CATTLE_LOCAL_ID
        if(y1<last_y1+70 and y2<last_y2+70):
            CATTLE_LOCAL_ID-=1
            #print('skipped second here')
            for i in range(len(STORED_MID_Y)):
                STORED_MISS[i]=5
            return -1
            
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
          
    result = []
    result.append(str(id-1))
    
   
    removed = 0
   
                 
    return result

In [12]:

async def HOKKAIDO_TRACKING(y,h,id,cow_srno=1):
    global STORED_IDS
    global STORED_MID_Y
    global STORED_MID_Y1
    global STORED_MID_Y2
    global STORED_MISS
    global LAST_SEEN_IDS
    global LAST_SEEN_ID_CENTROIDS
    global CATTLE_LOCAL_ID
    global IS_FIRST_CATTLE 
    y1 , y2 = y , y+h
    
    if IS_FIRST_CATTLE:
        IS_FIRST_CATTLE = False
        id = CATTLE_LOCAL_ID
    
    mid_y = int(2*y + h)/2
    IS_NEW = True
    last_id = 999
    last_y1 = 0
    last_y2 = 0
    if(len(STORED_IDS)>0): 
        last_id = STORED_IDS[len(STORED_IDS)-1]
        last_y1 = STORED_MID_Y1[len(STORED_MID_Y1)-1]#max(STORED_MID_Y1)
        last_y2 = STORED_MID_Y2[len(STORED_MID_Y2)-1]#max(STORED_MID_Y2)
        MISSED_LEN = len(STORED_MISS)
    
        removed = 0
        for i in range(MISSED_LEN):
    
            missed = STORED_MISS[i-removed]
    
            if(missed>70): #if missed 35 frames
    
                del STORED_MISS[i-removed]  
                del STORED_MID_Y[i-removed]
                del STORED_MID_Y1[i-removed]
                del STORED_MID_Y2[i-removed]
                del STORED_IDS[i-removed]
                removed+=1
                #print('removed')
                
    #clear misses
   
    
    threshold_1 = 180 #200 #300
    threshold_2 = 230 #250  #230
    Distance = 1000
    
    for i in range(1,len(STORED_MID_Y)+1):
    
        if(STORED_MID_Y1[-i]-threshold_1<=y1 and STORED_MID_Y1[-i]+threshold_1-50>=y1) or (STORED_MID_Y2[-i]-threshold_2<=y2 and STORED_MID_Y2[-i]+threshold_2-50>=y2): # and IS_NEW): #previous 150 #200
            if(IS_NEW):
              
                
                Distance = abs(STORED_MID_Y1[-i] - y1)
                if(abs(STORED_MID_Y2[-i] - y2)<Distance):
                    Distance = abs(STORED_MID_Y2[-i] - y2)
                IS_NEW = False
                STORED_MID_Y1[-i] = y1
                STORED_MID_Y2[-i] = y2
                
                STORED_MISS[-i]=1
                id= STORED_IDS[-i]
    
            elif Distance >50:
                STORED_MISS[-i]+=1
            
        else:
            STORED_MISS[-i]+=1    
     
    if(IS_NEW == False and id<last_id):
        if(y1>=last_y1 and y2>=last_y2):
            IS_NEW = True
        elif (y1>=last_y1-100 or y1<=last_y1+100) and (y2>=last_y2-100 or y2<=last_y2+100):
            return -1
    
        #elif(cow_srno==1):
    if(IS_NEW):
        CATTLE_LOCAL_ID+=1
        id=CATTLE_LOCAL_ID
        if(y1<last_y1-50 and y2<last_y2-50):
            CATTLE_LOCAL_ID-=1
            print('skipped')
            for i in range(len(STORED_MID_Y)):
                STORED_MISS[i]=5
            return -1
            
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
    
    if(IS_NEW) and False:
    
        updatedID = Is_Duplicate_Id(y1,y2,id)
        if(int(last_id) <int(updatedID) and y1<last_y1-150 and y2<last_y2-150): # duplicate cattle with increased cattleID
            CATTLE_LOCAL_ID-=1
            for i in range(len(STORED_MID_Y)-1,0,-1):
                STORED_MISS[i]=15
            return -1
        if(int(last_id)-1>int(updatedID)):
            return -1
        
        id=CATTLE_LOCAL_ID
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
          
    result = []
    result.append(str(id-1))
    removed = 0
    for i in range(len(STORED_MID_Y)-1,0,-1):
        if(y1>STORED_MID_Y1[i] and y2>STORED_MID_Y2[i]):
            del STORED_MISS[i-removed]  
            del STORED_MID_Y[i-removed]
            del STORED_MID_Y1[i-removed]
            del STORED_MID_Y2[i-removed]
            del STORED_IDS[i-removed]
            removed+=1
                 
    return result

In [13]:
mapping_data = {}
def mapper():
    global mapping_data
    mapping_data = {}
    csv_file = 'D:\\Python\\SULarbmon\\Python\\env\\yolov8_june\\ultralytics\\runs\\segment\honkawa_3fps\\fill_black_dec_20235\\1103_blackfill_mapping.csv'

# Create a dictionary to store the data
    

    # Read the CSV file and populate the dictionary
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        #header = next(reader)  # Assuming the first row contains column headers

        for row in reader:
            cattle_id = row[0]
            correct_id = int(row[1])
            mapping_data[cattle_id] = correct_id

#mapper()

In [13]:
UNKNOWN_THRESHOLD = 0.5
cattle_track_count = 0
def CALCULATE_MAX_CATTLE_ID(csv_path):
    print(csv_path, " is csv_path and ")
    global cattle_track_count
    cattle_track_count+=1
    data = pd.read_csv(csv_path)
    
    list_of_csv = [list(row) for row in data.values]

    prev_id_record = [] 
    prev=None

    current_cow = []
    excel_cow_count = []
    #boxes = []
    #file_locations = []
    black_count = (data['COLOR'] == 'B').sum()
    #print((testing['RANK1'] == 10172).sum())
    #total_cattlelen(testing['RANK1']))
    total_cattle_count = len(list_of_csv)
    if total_cattle_count/2 < black_count:
        return 'BLACK'
    
    
    # Group by 'id' and sum the 'score' for each group
    grouped_df = data.groupby('RANK1')['RANK1_prob'].agg(['sum', 'count']).reset_index()

    # Get the two highest occurrences
    print("Processing tracking :",cattle_track_count)
    distinct_count = data['RANK1'].nunique()
    
        
    top_two_occurrences = grouped_df.nlargest(2, 'sum')
    first_id, first_score, first_count = top_two_occurrences.iloc[0]
    if distinct_count ==1 :
        return int(first_id)
    if first_count < 7: #and distinct_count > 15 and:
        second_ranks = data['RANK2'].value_counts()
        max_occurrence_count = second_ranks.max()
        if max_occurrence_count < 7:
            return -1
        # Get the value with the maximum occurrence
        return second_ranks.idxmax()
        
        
        # Get the maximum occurrence count
        
        #return -1 # UNKNOWN
    second_id, second_score, second_count = top_two_occurrences.iloc[1]
    print("The two highest occurrences: ")
    print(f"First id: {first_id}, Score: {first_score}, Count: {first_count}")
    print(f"Second id: {second_id}, Score: {second_score}, Count: {second_count}")
    #if second_score/first_score > 0.8 and second_count/first_count > 0.8: #similar so consider as unknown
    #    return -1 #UNKNOWN
    return int(first_id)
    #for i in range (total_cattle_count):
    #    filtered_id = list_of_csv[i][0]
    #    actual_id = list_of_csv[i][2]
        
    #    try: 
    #        index = current_cow.index(actual_id)
    #        excel_cow_count[index]+=1
    #    except:
    #        current_cow.append(actual_id)
    #        excel_cow_count.append(1)

    #maxpos = excel_cow_count.index(max(excel_cow_count)) # max position in cow_count
    #cattle_id = current_cow[maxpos]  # fetch the cattle id from max position index because they are same index
    #print('threshold of tracking id:', cattle_track_count , ' is ' , max(excel_cow_count) / total_cattle_count )
    
    #if(max(excel_cow_count) / total_cattle_count < UNKNOWN_THRESHOLD):
    #    return -1 #Unknown
    #return int(cattle_id)#,file_locations,boxes


In [14]:

def get_final_cattle_id(save_dir,total_cattle):
    final_id = []
    global UNKNOWN_CATTLE
    global BLACK_CATTLE 
    for i in range(1,total_cattle+1):
        csv_path = save_dir + "/" + str(i) + "/" + str(i) + ".csv"
        
        output_id =  CALCULATE_MAX_CATTLE_ID(csv_path)
        print(output_id)
        if output_id == 'BLACK':
            BLACK_CATTLE +=1
            #output_id = get_black_cattle_id(save_dir+"/"+str(i))
            #BLACK_CATTLE+=1
            output_id = "black_"+str(BLACK_CATTLE)
            print('tracking ',i, ' is black cow ',output_id)
        if output_id == -1:
            UNKNOWN_CATTLE+=1
            output_id = "U_"+str(UNKNOWN_CATTLE)
            print('tracking ',i, ' is ',output_id)
            
        final_id.append(output_id)
    return final_id

In [15]:
async def get_black_cattle_id(filePath):
    predictedId = []
    for img_path in glob.glob(os.path.join(filePath, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        labels = await HOKKAIDO_B_SVM(img)
        predictedId.append(labels[0])
    
    
    return find_max_occurrence(predictedId)
    

In [16]:

from collections import Counter

def find_max_occurrence(arr):
    # Use Counter to count occurrences of each element
    counter = Counter(arr)

    # Find the element with the maximum occurrence
    max_element = max(counter, key=counter.get)

    # Get the count of the maximum occurring elemen

    return max_element

In [17]:
import glob

def writeVideo(cap,filePath,csv_name,cattle_ids,total_frame):
    img_array = []
    size = (1920,1080)
    names = ['cow']
    main_csv_index = 0
    data = pd.read_csv(filePath+"/"+csv_name)

    list_of_csv = [list(row) for row in data.values]
    vid_name = os.path.basename(os.path.normpath(filePath))
    total_count = len(list_of_csv)
    prev_image_id = 1
    #vid_path = str(Path(filePath + "/" + vid_name ).with_suffix('.mp4'))
    #out = cv2.VideoWriter(vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 6, size)
    #if len(img_locations)<10: 
    #    return -1
    for filename in glob.glob(filePath+"/all_images/"+'/*.jpg'):
        img = cv2.imread(filename)
        
        #print(os.path.isfile(filename))
        #print(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    image_location = filePath+"/all_images/";
    img_index = 0
    for ind in range(1,total_frame):
        image_path = image_location + str(ind) +".jpg"
        #print(image_path)
        img = cv2.imread(image_path)
        
        #print(os.path.isfile(filename))
        #print(filename)
        #print(filename)
        #height, width, layers = img.shape
        #size = (width,height)
        #img = img_array[img_index]
        #img_index +=1
        #img = cv2.resize(img, size, interpolation = cv2.INTER_AREA)
        #print(img.Shape)
        #cv2.imshow('Cattle Images ', img)
        #if cv2.waitKey(1) & 0xFF == ord(' '):
        #    break
        annotator = Annotator(img, line_width=3, example=str(names)) #font here
        from_index = main_csv_index
        print(from_index, ' from index')
        is_draw = False
        for i in range(from_index,total_count-1):
            
            main_csv_index+=1
            print(ind, "  <--x-->  ", list_of_csv[i][0])
            if(ind!=list_of_csv[i][0]):
                main_csv_index-=1
                break
            print('SAVING!!!!!')
            tracking_id  = list_of_csv[i][1] -1
            xyxy = [list_of_csv[i][2],list_of_csv[i][3],list_of_csv[i][4],list_of_csv[i][5]]

            try:
                annotator.box_label(xyxy,str(cattle_ids[tracking_id]), color=(15, 0, 255))
            except:
                annotator.box_label(xyxy,cattle_ids[tracking_id]+"_"+str(tracking_id), color=(15, 0, 255))
            is_draw = True
        if is_draw:
            annotated_img =cv2.resize(annotator.result(),size) 
            cap.write(annotated_img)
            #except:
            #    print('did not write')
            #    continue
    #ut.release()
    img_array=[]
    print("done ", vid_name)
    #v2.destroyAllWindows()
    return id


In [18]:
# Function to draw a bounding box and annotate the image
def draw_bounding_box(image, box, label):
    # Extract the coordinates from the box
    x1, y1, x2, y2 = box
    #print(x1,' ',y1,x2,y2)

    # Draw the bounding box rectangle on the image
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Define the text properties
    text = f'{label}'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale =1
    thickness = 2

    # Calculate the size of the text
    (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)

    # Calculate the position for placing the text
    text_x = x1
    text_y = y1 - 10 if y1 >= 20 else y1 + 10 + text_height

    # Draw the text background rectangle
    cv2.rectangle(image, (text_x, text_y - text_height - 5), (text_x + text_width, text_y), (0, 255, 0), -1)

    # Put the label text on the image
    cv2.putText(image, text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness, cv2.LINE_AA)

# Example usage



In [19]:
def overlay(image, mask, color, alpha, resize=None):
    """Combines image and its segmentation mask into a single image.
    
    Params:
        image: Training image. np.ndarray,
        mask: Segmentation mask. np.ndarray,
        color: Color for segmentation mask rendering.  tuple[int, int, int] = (255, 0, 0)
        alpha: Segmentation mask's transparency. float = 0.5,
        resize: If provided, both image and its mask are resized before blending them together.
        tuple[int, int] = (1024, 1024))

    Returns:
        image_combined: The combined image. np.ndarray

    """
    # color = color[::-1]
    colored_mask = np.expand_dims(mask, 0).repeat(3, axis=0)
    colored_mask = np.moveaxis(colored_mask, 0, -1)
    masked = np.ma.MaskedArray(image, mask=colored_mask, fill_value=color)
    image_overlay = masked.filled()

    if resize is not None:
        image = cv2.resize(image.transpose(1, 2, 0), resize)
        image_overlay = cv2.resize(image_overlay.transpose(1, 2, 0), resize)

    image_combined = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)

    return image_combined

In [20]:
def GET_LEFT_TO_RIGHT_ORDER(x1s):
    #return x1s
    orders = []
    size = len(x1s)
    #print(' x1s     ',x1s)
    clones = x1s[:]
    for i in range(size):
        #print(clones)
        id = (x1s.index(min(clones)))
        #print(id)
        orders.append(id)
        clones.pop(clones.index(min(clones)))
    #print(' sorted order :',orders)
    return orders

In [21]:
async def add_lines_to_excel(existing_file, data_to_add):
    
    try:
        df_existing = pd.read_csv(existing_file,dtype={
    'ImageId': 'string',
    'LocalId': np.int64,
    'xyxy1': np.int64,
    'xyxy2': np.int64,
    'xyxy3': np.int64,
    'xyxy4': np.int64
})
    except FileNotFoundError:
        df_existing = pd.DataFrame()

    df_to_add = pd.DataFrame(data_to_add)
    df_combined = pd.concat([df_existing, df_to_add], ignore_index=True)
    df_combined.to_csv(existing_file, index=False)


#add_lines_to_excel(existing_excel_file, data_to_add)
#This function encapsulates the process of adding lines to an existing Excel file using a DataFrame. It first attempts to read the existing Excel file, creating an empty DataFrame if the file doesn't exist. Then, it creates a DataFrame from the data to add and concatenates it with the existing DataFrame. Finally, it writes the combined DataFrame back to the Excel file.

#Make sure to replace 'path/to/your/existing/excel/file.xlsx' with the actual path to your existing Excel file and adjust the data_to_add dictionary accordingly.







In [22]:
def create_default_csv(csv_file_path):
    default_columns = ['ImageId', 'LocalId', 'xyxy1', 'xyxy2', 'xyxy3', 'xyxy4']  # Add your desired column headers here

    # Create an empty DataFrame with the default columns
    empty_df = pd.DataFrame(columns=default_columns)

    # Save the empty DataFrame to a CSV file
    empty_df.to_csv(csv_file_path, index=False)

In [23]:
def concat_main_csv(data):
    with main_df_lock:
        global MAIN_DF
        MAIN_DF = pd.concat([MAIN_DF, data], ignore_index=True)

In [24]:
#creating default csv
#filename of csv path, N_TOP for number of TOP value
def creating_default_csv(csv_file_name,N_TOP):
    data = [
    ["ID","COLOR",	"RANK1", "RANK1_prob"]
    ]
    
    for i in range(2,N_TOP+1):
        data[0].extend([f"RANK{i}",f"RANK{i}_prob"])
    print(data)
                       
    # Specify the file name

    # Writing to the CSV file
    with open(csv_file_name, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header
        writer.writerow(data[0])

In [25]:
def write_to_csv(file_name, data):
    with open(file_name, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

In [26]:
def calculate_black_white_percentage_rgb(image, threshold=150):
    # Apply the threshold to each channel
    binary_mask = (image <= threshold).all(axis=-1) * 255
    
    #cv2.imshow('Masked Image', image)
    #cv2.waitKey(1)
    
        
    # Calculate the percentage of black and white pixels
    total_pixels = binary_mask.size
    black_pixels = np.count_nonzero(binary_mask == 0)
    white_pixels = np.count_nonzero(binary_mask == 255)

    black_percentage = (black_pixels / total_pixels) * 100
    white_percentage = (white_pixels / total_pixels) * 100

    return black_percentage, white_percentage



In [27]:
async def calculate_white_pixel_percentage(image):
    # Convert the image to grayscale
    
    if not image.dtype in [np.uint8, np.uint16]:
        image = (image / image.max() * 255).astype(np.uint8)
    #cv2.imshow('BGR Image', image)
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #thresholded_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    #white_pixel_count = np.sum(thresholded_image == 255)
    #total_pixels = thresholded_image.size
    #threshold_image = otsu_threshold(gray_image)
    #otsu_thresholded_image = otsu_threshold_dynamic(gray_image)
    # Apply Otsu thresholding
    #otsu_thresholded_image = otsu_threshold(gray_image)

    # Count white and black pixels after Otsu thresholding
    #white_pixel_count = np.sum(otsu_thresholded_image == 255)
    #black_pixel_count = np.sum(otsu_thresholded_image == 0)

    # Calculate the total number of pixels
    #total_pixels = otsu_thresholded_image.size

    # Calculate the white pixel percentage
    #white_pixel_percentage = (white_pixel_count / total_pixels) * 100
    

    # Calculate the white pixel percentage
    #white_pixel_percentage = (white_pixel_count / total_pixels) * 100
    #print('White Pixel Percentage is :',white_pixel_percentage)
    #return white_pixel_percentage
    #_, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    #threshold_value = otsu_threshold(gray_image)
    #threshold_value = find_peak_intensity(gray_image)
    #threshold_value = otsu_CDF(gray_image)
    #return dynamic_threshold_cattle_back(gray_image)
    return dynamic_threshold_cattle_back_area(gray_image)

    #print(f"Otsu's threshold: {threshold_value}")

    # Plot the histogram
    if True:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(gray_image, cmap='gray')
        plt.title('Original Image')

        plt.subplot(1, 2, 2)
        plt.hist(image.flatten(), 256, [0, 256], color='r')
        plt.axvline(x=threshold_value, color='k', linestyle='-.', linewidth=2, label=f'Otsu\'s Threshold: {threshold_value}')
        plt.title('Histogram')
        plt.legend()

        plt.show()

    # Apply thresholding
    

    # Display the original and thresholded images
    #cv2.imshow('Original Image', image)
    #cv2.imshow('Thresholded Image', thresholded_image)
    #cv2.imshow('gray Image', gray_image)
    
    # Apply threshold to create a binary mask
    _, binary_mask = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Calculate the percentage of white pixels
    total_pixels = binary_mask.size
    white_pixels = np.count_nonzero(binary_mask == 255)

    white_percentage = (white_pixels / total_pixels) * 100
    white_pixels_only = cv2.bitwise_and(image, image, mask=binary_mask)

    # Display the images
    cv2.imshow('Original Image', image)
    cv2.imshow('White Pixels Only', white_pixels_only)

    #return white_pixel_percentage
    return white_percentage


In [28]:
def dynamic_threshold_cattle_back(img, block_size=15, constant=5):
    # Read the image
    

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, constant)

    # Display the original and thresholded images
    cv2.imshow('Original Image', img)
    cv2.imshow('Thresholded Image', thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #return thresh

In [29]:
def otsu_CDF(image):
    start = 50
    end = 230
    

    
    # Ensure the image is in the desired range (30 to 230 pixels)
    image = np.clip(image, 30, 230)

    # Step 2: Calculate histogram
    histogram, bins = np.histogram(image.flatten(), bins=256, range=[30, 231])

    # Step 3: Normalize histogram
    normalized_histogram = histogram / np.sum(histogram)

    # Step 4: Calculate CDF and Mean
    cdf = np.cumsum(normalized_histogram)
    #mean_intensity = np.sum((np.arange(30, 231) * normalized_histogram)[bins[:-1].astype(int)])


    # Step 5: Calculate Between-Class Variance
    variances = np.zeros(201)  # 230 - 30 + 1 = 201
    for i in range(1, 201):
        w0 = np.sum(normalized_histogram[:i])
        w1 = np.sum(normalized_histogram[i:])
        mu0 = np.sum(np.arange(30, 30 + i) * normalized_histogram[:i]) / w0
        mu1 = np.sum(np.arange(30 + i, 30 + i + len(normalized_histogram[i:])) * normalized_histogram[i:]) / w1
        variances[i] = w0 * w1 * (mu0 - mu1)**2

    # Step 6: Find Optimal Threshold within the specified range
    optimal_threshold = np.argmax(variances) + 30  # Add back the lower bound of the range
    
    return optimal_threshold

In [30]:

def otsu_threshold(image):
    #cv2.imshow('BGR Image', image)
    
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Compute histogram and normalized histogram
    hist, bins = np.histogram(image.flatten(), 256, [0, 256])
    
    #peak_intensity = np.argmax(hist)

    #Testing peak_intensity
    #return peak_intensity-5
    
    hist_norm = hist / hist.sum()

    # Initialization
    sum_total = 0
    for i in range(256):
        sum_total += i * hist_norm[i]

    sum_background = 0
    weight_background = 0

    max_variance = 0
    threshold = 0

    # Iterate through all possible thresholds
    for i in range(256):
        weight_background += hist_norm[i]
        weight_foreground = 1 - weight_background

        sum_background += i * hist_norm[i]

        mean_background = sum_background / (weight_background + 1e-10)
        mean_foreground = (sum_total - sum_background) / (weight_foreground + 1e-10)

        intra_class_variance = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2

        # Update threshold if variance is greater
        if intra_class_variance > max_variance:
            max_variance = intra_class_variance
            threshold = i

    return threshold

# Example usage:



In [31]:
def otsu_threshold_dynamic(image):
    _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded_image


In [32]:
def find_peak_intensity(image, intensity_range=(50, 255)):
    # Clip the image values to the specified range
    clipped_image = np.clip(image, intensity_range[0], intensity_range[1])

    # Compute histogram and find the peak intensity within the specified range
    hist, bins = np.histogram(clipped_image.flatten(), intensity_range[1] - intensity_range[0] + 1, [intensity_range[0], intensity_range[1]])
    peak_intensity = np.argmax(hist) + intensity_range[0]

    return peak_intensity

In [33]:
from PIL import Image, ImageOps

def resize_image(input_array,new_size):
    # Open the image
    #new_size = (224,224)
    original_image = Image.fromarray(input_array)

    # Create a new image with the desired size and fill it with black pixels
    new_image = Image.new("RGB", new_size, (0, 0, 0))

    # Calculate the position to paste the original image in the center
    x_offset = (new_size[0] - original_image.width) // 2
    y_offset = (new_size[1] - original_image.height) // 2

    # Paste the original image onto the new image
    new_image.paste(original_image, (x_offset, y_offset))
    #return cv2.cvtColor(np.array(new_image),cv2.COLOR_RGB2BGR)
    return np.array(new_image)
    #bgr_array = cv2.cvtColor(numpy_array, cv2.COLOR_RGB2BGR)
    # Save the result_image
    #new_image.save(output_path)





In [34]:
def dynamic_threshold_cattle_back(img, threshold_factor=0.75): #0.7 was best
    # Read the image

    # Calculate a dynamic threshold based on pixel intensity
    threshold_value = int(np.max(img) * threshold_factor)

    # Apply the threshold
    _, thresh = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    total_pixels = img.size
    white_pixels = cv2.countNonZero(thresh)
    white_pixel_percentage = (white_pixels / total_pixels) * 100
    
    
    # Display the original and thresholded images
    #cv2.imshow('Original Image', img)
    #cv2.imshow('Thresholded Image', thresh)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return white_pixel_percentage

In [35]:
def dynamic_threshold_cattle_back_area(img, threshold_factor=0.7, min_area=10): #min_area= 200 before
    # Read the image


    # Calculate a dynamic threshold based on pixel intensity
    threshold_value = int(np.max(img) * threshold_factor) - 5
    
    # Apply the threshold
    _, thresh = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Morphological operations to eliminate small, straight white regions
    #kernel = np.ones((5, 5), np.uint8)
    if False:
        kernel = np.ones((7, 7), np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # Find contours and filter based on minimum area
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < min_area:
                cv2.drawContours(thresh, [contour], -1, (0, 0, 0), thickness=cv2.FILLED)

    # Calculate the percentage of white pixels
    total_pixels = img.size
    white_pixels = cv2.countNonZero(thresh)
    white_pixel_percentage = (white_pixels / total_pixels) * 100

    # Display the original and processed images
    cv2.imshow('Original Image', img)
    cv2.imshow('Processed Image', thresh)
    global image_count
    original_path = Path(base_path+ '/Thresh/' + f'{image_count}_original.jpg')
    thresh_path = Path(base_path+ '/Thresh/' + f'{image_count}_thresh.jpg')
    cv2.imwrite(str(original_path), img) #save ori_img
    cv2.imwrite(str(thresh_path), thresh) #save ori_img
    return white_pixel_percentage


In [36]:
def DETECTOR(farmName):
    if farmName == HONKAWA:
        return YOLO("D:/Python/SULarbmon/Python/env/yolov8_june/runs/segment/Honkawa_Weights_v1/weights/best.pt")  # load a custom model
    else:
        return YOLO('runs/segment/train4/weights/best.pt')   # load a custom model

In [37]:
#mainfile
thread_pool = []
save_threads = []
BLACK_CATTLE =0

X1=200 #same as NEW_BLACK_X1
X2=400 #same as NEW_BLACK_X2 # incase of x2 out of bound
Y1=120
Y2=500

Y1_NEW=120 #125  #decrease here to extend, increase to shrink 
Y2_NEW=510  #500  # redyce here to extend , increase to do vice casa 460 previous
FRAME = 1

default=640

BATCH = 100
BATCH_COUNT = 1
PREV_BATCH = 0

LAST_SEEN = time.time()
FIRST_SEEN = True
demo_img_save_path = []
main_df_lock = threading.Lock()


#end
prevId_record =[]
MAX_prevId = [] 
MAX_xyxy1 = [] 
MAX_xyxy2 = [] 
MAX_xyxy3 = [] 
MAX_xyxy4 = [] 
MAX_orgId = []
MAX_org_prod = []
RANK_2 = []
RANK_3 = []
RANK_4 = []
IMAGE_STORED_LOCATION = []
IMAGE_ID_LIST = []
LOCAL_ID_LIST = []
#end
#prevId_record =[]
TOTAL_CATTLE_COUNT = 0 

N_TOP = 2 # top predicted value 

#end

#region Cattle Tracking 
STORED_IDS= []
STORED_MID_Y = []
STORED_MID_Y1 = []
STORED_MID_Y2 = []
STORED_MISS = []
PREVIOUS_ID = [] # keep the record of last seen ids and position
PREVIOUS_Y1 = [] 
PREVIOUS_Y2 = [] 
PREVIOUS_LOCAL_IDS = []
CATTLE_LOCAL_ID= 1
IS_FIRST_CATTLE = True

UNKNOWN_CATTLE = 0
start_time = time.time()

MAIN_DF_COLUMNS = ['ImageId',	'LocalId',	'xyxy1',	'xyxy2',	'xyxy3',	'xyxy4'] # using to store all cattle information in here instead of list
MAIN_DF = pd.DataFrame(columns = MAIN_DF_COLUMNS)

# Predict with the model
project = 'D:/Python/SULarbmon/Python/env/yolov8_june/ultralytics/runs/segment/HOKKAIDO_PAPER/'
name = 'HOKKAIDO_BWU'
#dataset = "D:\\815_CowDataChecking\\Honkawa\\2023-07-31\\03\output_1fps\\20230731-030500-031000_10fps_1fps.mp4" //HONKAWA
dataset = "D:\\815_CowDataChecking\\20221230\\20221230_E_cow\\20221230_152538_8796.mkv"
save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_track')  #open this when running single video
#save_vid_name = dataset.split("\\")[-1]+"_track" # open this when running multiple videos

print(save_vid_name)
#results = model(dataset,imgsz=(960,640),save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.3)
#results = model('D:\\815_CowDataChecking\\20221228\\20221228_E_cow\\20221228_151533_D474.mkv',imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.2)
#save_dir = increment_path(Path(project) / name, exist_ok=True)  # increment run
save_dir = increment_path(Path(project) / name,mkdir=True)
csv_main_file_path = str(save_dir) + "\main_csv.csv"
#create_default_csv(csv_main_file_path)
#(save_dir / 'labels' if False else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
manual_cow_count = 0

#cap.set(4, 480)
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
print(save_vid_path)
cap = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 5, (1088,1088))
skip_count = 2
skip_track = 1
image_count = 1

SAVING_THRESHOLD =200
SAVED_COUNT = 1

CATTLE_SAVING_THRESHOLD = 15
CATTLE_SAVED_COUNT = 0
last_tracking_id = 0
base_path =''


HOKKAIDO = "HOKKAIDO"
HONKAWA = "HONKAWA"
FARMNAME = HOKKAIDO
model = DETECTOR(FARMNAME)
#results = model(dataset,imgsz=(960,640),save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.3) if FARMNAME == HONKAWA else 
results = model(dataset,imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.3) #HOKKAIDO
           
skip_track = 0
for result in results:
    if(skip_track==2):
        skip_track = 0
        continue
    skip_track+=1
    
    #print(result.boxes)
    vid_path = result.path
    filename = vid_path.split("\\")[-1].replace(".mp4","")
    
    boxes = result.boxes.cpu().numpy()
    detections = boxes.xyxy.tolist()
    #print(detections)
    # Sort the detections based on the x1 coordinate (i.e., left-to-right)
    #detections.sort(key=lambda x: x[0])
    left_to_right = GET_LEFT_TO_RIGHT_ORDER([t[0] for t in detections])
    #print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    #print(left_to_right)
    #print(detections)
    ori_img = result.orig_img
    if FARMNAME == HOKKAIDO:
        ori_img = cv2.resize(ori_img, (1088,1088), interpolation = cv2.INTER_AREA)

    else:
        ori_img = cv2.resize(ori_img, (640,384), interpolation = cv2.INTER_AREA)
    annotator = Annotator(ori_img)
    box_count = 0
    cow_position = 1
    
    h,w = result.orig_shape
    count = 1
    b_boxes = []
    masks = []
    ids = []
    has_cattle = False
    
    if  result.masks != None:
        
        #result_masks = result.masks.cpu().numpy().masks.astype(bool)
        #for m in result.masks.cpu().numpy().masks.astype(bool):
        result_masks = result.masks.cpu().numpy().masks.astype(bool)
        
        for LR_index in left_to_right:
            m = result_masks[LR_index]
        #for i in range(1,len(result_masks)+1):
        #for m in result.masks.masks.astype(bool):
            
            xyxy = boxes[LR_index].xyxy[0]
            #print('LR  INDEX  ',LR_index)
            #print(xyxy)
        #   m = result_masks[-1]
            #print(xyxy)
            #print(m.shape)
            box_count += 1
            x1= int(xyxy[0])
            y1= int(xyxy[1])
            x2= int(xyxy[2])
            y2= int(xyxy[3])
            #print(xyxy)
            #print(m.shape)
            
            ################## Validate  #####################
            if(FARMNAME == HOKKAIDO):
                if(HOKKAIDO_ROI(x1,y1,x2,y2,h,w)==False):
                    #print("I was skipped")
                    continue
            elif(HONKAWA_ROI(x1,y1,x2,y2,h,w)==False):
                    #print("I was skipped")
                    continue
            #print("not skipped")
            has_cattle = True
            #new_results.append(result)
            box_left = x1
            box_top = y1
            box_w = x2 - x1
            box_h = y2 - y1

            new = np.zeros_like(ori_img, dtype=np.uint8)
            new[m] = ori_img[m]
            
            if FARMNAME == HOKKAIDO:
                x1= int(x1 * (1088/2992))
                x2= int(x2 * (1088/2992))
                y1= int(y1 * (1088/2992))
                y2= int(y2 * (1088/2992))
            else :
                x1= int(x1 * (640/1920))
                x2= int(x2 * (640/1920))
                y1= int(y1 * (384/1080))
                y2= int(y2 * (384/1080))

            crop = new[y1:y2, x1:x2]
            #cv2.imshow("Original Crop ",crop)
            high = y2-y1
            white_percentage = await calculate_white_pixel_percentage(crop)
            crop = resize_image(crop, (high,high))
            if FARMNAME == HOKKAIDO:
                #img = cv2.resize(crop, (SIZE_224, SIZE_224))
                img = resize_image(crop, (SIZE_224,SIZE_224))
            elif FARMNAME == HONKAWA:
                img = resize_image(crop, (SIZE_112,SIZE_112))

            #cv2.imshow("Resize Crop ",crop)
            
            img=img / 255.0
            
            ############# LABLE
            
            
            
            ###### LABEL
            #img = cv2.resize(crop, (SIZE, SIZE))
            #img=img / 255.0
            #print(box_left,'    xxxxxx     ' ,box_w)
            prev_id = await HOKKAIDO_TRACKING(box_top,box_h,1,cow_position) ## just passing x values instead of yprev_id = await HOKKAIDO_TRACKING(box_left,box_h,1,cow_position) ## just passing x values instead of y
            #########################################  
            HAS_COW=True
            #has_cattle = True
            if(prev_id==-1): #skip cattle when prev_id // filter id is -1
                if(count==1):
                    has_seen_cattle=False
                    count-=1
                #print('skipped')
                continue
            
            
            print(f"White Percentage of {int(prev_id[0])}: {white_percentage:.2f}%")
            cattle_color  = 'W'
            #if white_percentage < 0.6 : #0.8 old value
            if white_percentage < 1.5 : #0.8 old value
                cattle_color  = 'B'
                ids.append(str(prev_id[0])+'_Black')
            else:
                ids.append(prev_id[0])
            t_start = time.time()
            if cattle_color == 'B':
                labels = await HOKKAIDO_B_SVM(img)
            else:
                labels = await HOKKAIDO_NB_SVM(img)
            print("identification used ",t_start-time.time())

            #print(labels)
            labels.insert(0,cattle_color)
            labels.insert(0,prev_id[0])
            
            has_cattle = True
            
            
            last_tracking_id = int(prev_id[0])
            masks.append(m)
            b_boxes.append([x1,y1,x2,y2])
            #print(prev_id)
            cow_position+=1 
            BATCH_COUNT = prev_id[0] # skip batch count here  
            
            base_path = str(Path(save_dir / prev_id[0]))
            default_csv_path = base_path +"/"+prev_id[0]+".csv"
            if not os.path.exists(base_path):
                os.makedirs(base_path)
                
                creating_default_csv(default_csv_path,N_TOP)

            demo_annotated_img_save_path = Path(base_path+ '/' + f'{image_count}.jpg')
            cv2.imwrite(str(demo_annotated_img_save_path), crop) #save crop
            CATTLE_DATA = {}
            #is_new_cattle = False
            try:
                
                index_prevId = prevId_record.index(int(prev_id[0]))
                
                #is_new_cattle = True
            except :
                TOTAL_CATTLE_COUNT +=1 
                print('new cattle  xxxxxxxxxxxxxxxxxxxxx new cattle alert ' )    
                prevId_record.append(int(prev_id[0]))
            
            
            # WRITING CSV FOR EACH CATTLE
            
            write_thread = threading.Thread(target=write_to_csv, args=(default_csv_path,labels))
            write_thread.daemon = True  # Set the thread as daemon
            write_thread.start()
            thread_pool.append(write_thread)
            
            
                
            
            IMAGE_ID_LIST.append(image_count)
            LOCAL_ID_LIST.append(int(prev_id[0]))
            MAX_xyxy1.append(x1)
            MAX_xyxy2.append(y1)
            MAX_xyxy3.append(x2)
            MAX_xyxy4.append(y2)
            

            manual_cow_count += 1

            manual_cow_count += 1

    #frame = annotator.result() 
   
    
    
    frame = cv2.resize(ori_img, (1088,1088), interpolation = cv2.INTER_AREA)
    
    if has_cattle:  
        #print("total saved cattle count -=>",CATTLE_SAVED_COUNT)
        data_to_add = {
        'ImageId': IMAGE_ID_LIST,
        'LocalId': LOCAL_ID_LIST,
        'xyxy1': MAX_xyxy1,
        'xyxy2': MAX_xyxy2,
        'xyxy3': MAX_xyxy3,
        'xyxy4': MAX_xyxy4
        # Add more columns as needed
        }
        df_to_add = pd.DataFrame(data_to_add)
        #MAIN_DF = pd.concat([MAIN_DF,df_to_add], ignore_index=True)
        csv_thread = threading.Thread(target=concat_main_csv, args=(df_to_add,))
        csv_thread.start()
        
        base_path = str(Path(save_dir / 'all_images'))
        if not os.path.exists(base_path):
            os.makedirs(base_path)
            os.makedirs(base_path+'/Thresh')
        del IMAGE_ID_LIST[:]
        del LOCAL_ID_LIST[:]
        del MAX_xyxy1[:]
        del MAX_xyxy2[:]
        del MAX_xyxy3[:]
        del MAX_xyxy4[:]

        demo_annotated_img_save_path = Path(base_path+ '/' + f'{image_count}.jpg')
        cv2.imwrite(str(demo_annotated_img_save_path), ori_img) #save ori_img
        save_thread = threading.Thread(target=cv2.imwrite, args=(str(demo_annotated_img_save_path),ori_img))
        save_thread.daemon = True   
        save_thread.start()
        save_threads.append(save_thread)
        SAVED_COUNT += 1
        image_count += 1
        
        #ori_img = cv2.resize(ori_img,(1088,1088))
        for i in range(len(b_boxes)):
            box = b_boxes[i]
            mask = masks[i]
            #print("drawing")
            #print(ids)
            
            draw_bounding_box(ori_img,(box[0],box[1],box[2],box[3]),str(ids[i]))
            ori_img = overlay(ori_img,mask,(0,0,255),0.3)
        frame = cv2.resize(ori_img, (1088,1088), interpolation = cv2.INTER_AREA)
        
        video_thread = threading.Thread(target=cap.write(frame))
        video_thread.start()
        del ids[:]
        del b_boxes[:]
        del masks[:]
        
        if threading.active_count() > 200:
            print("Cleaning up threads")
            pass
        
            gc.collect()

            torch.cuda.empty_cache()
        
        
    cv2.imshow('YOLO V8 Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break
    
    
pass
        
cap.release() #tracking video
cv2.destroyAllWindows()
image_count -=1 

#while threading.active_count() > 1:


now=str(datetime.now().date())

IMAGE_STORED_LOCATION = []

cattle_ids = []
#################################\
manual_summarize_ids = []
manual_local_ids = []

manual_summarize_ids = []
manual_local_ids = []
#### write video after saving csv
final_cattle_count = 1

# SAVING MAIN CSV-
MAIN_DF.to_csv(csv_main_file_path, index=False)

MAIN_DF = None # CLEAR AFTER SAVING

save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_classification')  #open this when running single video
#save_vid_name=  dataset.split("\\")[-1].replace('.mp4','')+'_classification'

save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
print(save_vid_path)
classification_vid = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 5, (1088,1088))
print(save_vid_path)


#SAVING SUMMARIZE IDS
manual_summarize_ids = get_final_cattle_id(str(save_dir),TOTAL_CATTLE_COUNT)

summarize_id_csv = pd.DataFrame(manual_summarize_ids, columns = ["Cow Id"])

summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
#FINISH SAVING SUMMARIZE IDS

csv_name = 'main_csv.csv'
#region Write Video

try: 
    writeVideo(classification_vid,str(save_dir),csv_name ,manual_summarize_ids,image_count)
except:
    print(" I am ugly")
#summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
classification_vid.release()
cv2.destroyAllWindows()
end_time = time.time()


20221230_152538_8796_track
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37\20221230_152538_8796_track.mp4
yolo->engine->model.py93
yolo->v8->segement->model.py 235
I reach here. yolo->Enging->Predictor.py 98


I am in stream_loaders.py 168


video 1/1 (1/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 16.2ms
video 1/1 (2/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 13.0ms
video 1/1 (3/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 12.5ms
video 1/1 (4/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 4.9ms
video 1/1 (5/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 15.9ms
video 1/1 (6/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 16.7ms
video 1/1 (7/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 12.7ms
video 1/1 (8/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 (no detections), 

White Percentage of 1: 12.10%
1/1 [==============================] - 0s 135ms/step
VGG took  0.17329907417297363


video 1/1 (805/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 21.9ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.


...

SVM took  1.0472867488861084
LOOPING result took  0.0
identification used  -1.1281335353851318
White Percentage of 8: 12.26%
1/1 [==============================] - 0s 58ms/step
VGG took  0.07653641700744629


video 1/1 (1642/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 28.6ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/1 (1643/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 90.0ms


SVM took  1.0510516166687012
LOOPING result took  0.0
identification used  -1.1275880336761475


video 1/1 (1644/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 77.4ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/1 (1645/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 80.0ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/1 (1646/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 62.8ms
video 1/1 (1647/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 61.1ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/1 (1648/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 14.2ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/1 (1649/3933) D:\815_CowDataChecking\20221230\20221230_E_cow\20221230_152538_8796.mkv: 1088x1088 1 cow, 15.0ms
video 1/1 (1650/3933) D:\815_C

D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37\20221230_152538_8796_classification.mp4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37\20221230_152538_8796_classification.mp4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/1/1.csv  is csv_path and 
Processing tracking : 1
The two highest occurrences: 
First id: 4332.0, Score: 1.407945321228751, Count: 13.0
Second id: 3849.0, Score: 1.168010028868799, Count: 10.0
4332
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/2/2.csv  is csv_path and 
Processing tracking : 2
The two highest occurrences: 
First id: 4423.0, Score: 1.5072488178466694, Count: 9.0
Second id: 4149.0, Score: 1.1809365222067143, Count: 13.0
4423
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/3/3.csv  is csv_path and 
BLACK
t

In [ ]:

elapsed_time = end_time - start_time

print("Total Prediction Duration for 0304 AM: ",elapsed_time)
#run 

In [ ]:
cv2.destroyAllWindows()

In [ ]:

#SAVING SUMMARIZE IDS
manual_summarize_ids = get_final_cattle_id(str(save_dir),TOTAL_CATTLE_COUNT)

summarize_id_csv = pd.DataFrame(manual_summarize_ids, columns = ["Cow Id"])

summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
#FINISH SAVING SUMMARIZE IDS

csv_name = 'main_csv.csv'
#region Write Video
try:
    writeVideo(classification_vid,str(save_dir),csv_name ,manual_summarize_ids,image_count)
except:
    print(" I am ugly")
#summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
classification_vid.release()
cv2.destroyAllWindows()
end_time = time.time()

In [ ]:
for csv_index in range(len(MAX_prevId)):
    df = pd.DataFrame(MAX_prevId[csv_index] , columns = ['ID'])
    tracked_id = MAX_prevId[csv_index][0]
    try:
        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
        df["Original"] = org_ids
    except:
        df["Original"] = MAX_orgId[csv_index]
    now=str(datetime.now().date())

    save_csv_each_path = str(Path(save_dir / str(tracked_id) / f'{str(tracked_id)}.csv'))
    df.to_csv(save_csv_each_path, index= False)##asdfasdf
MAIN_DF.to_csv(csv_main_file_path, index=False)

In [ ]:
print(csv_main_file_path)
csv_main_file_path = 'D:\\Python\\SULarbmon\\Python\\env\\yolov8_june\\ultralytics\\runs\\segment\\honkawa\\291_unknown\\identification4\\main_csv.csv'
MAIN_DF.to_csv(csv_main_file_path, index=False)
print('save')

In [ ]:

project = 'D:\\Honkawa_Exe_output\\identification2_0805'
name = 'identification_new2'
dataset = "D:\\815_CowDataChecking\\Honkawa\\2023-08-04\\03\\0304" 
#save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_track')  #open this when running single video


#results = model('D:\\815_CowDataChecking\\20221228\\20221228_E_cow\\20221228_151533_D474.mkv',imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.2)
#save_dir = increment_path(Path(project) / name, exist_ok=True)  # increment run
save_dir = Path(project,name)
print(save_dir)

In [43]:
#save_dir = "D:\\Python\\SULarbmon\\Python\\env\\yolov8_june\\ultralytics\\runs\\segment\\honkawa\\1103\\identification_new2"
UNKNOWN_THRESHOLD = 0.5
cattle_track_count = 0
UNKNOWN_CATTLE = 0
BLACK_CATTLE =0


save_dir = "D:\\Python\\SULarbmon\\Python\\env\\yolov8_june\\ultralytics\\runs\\segment\\HOKKAIDO_PAPER\\HOKKAIDO_BWU37"
csv_main_file_path = save_dir + "\main_csv.csv"
dataset =  "D:\\815_CowDataChecking\\Honkawa\\2023-07-31\\03\\20221230_152538_8796.mp4" 
TOTAL_CATTLE_COUNT=8

now=str(datetime.now().date())

save_vid_name=  dataset.split("\\")[-1].replace('.mp4','')+'_classification'

save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
print(save_vid_path)
classification_vid = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 5, (1088,1088))
print(save_vid_path)
#### write video after saving csv
manual_summarize_ids = get_final_cattle_id(str(save_dir),TOTAL_CATTLE_COUNT)
summarize_id_csv = pd.DataFrame(manual_summarize_ids, columns = ["Cow Id"])
summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_50_percent_'+now+'.csv', index= False) 

csv_name = 'main_csv.csv'
#region Write Video
image_count = 207
try:
    writeVideo(classification_vid,str(save_dir),csv_name ,manual_summarize_ids,image_count)
except Exception as ex:
    print(ex)
#summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
classification_vid.release()
cv2.destroyAllWindows()


D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37\20221230_152538_8796_classification.mp4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37\20221230_152538_8796_classification.mp4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/1/1.csv  is csv_path and 
Processing tracking : 1
The two highest occurrences: 
First id: 4125.0, Score: 1.951537817, Count: 18.0
Second id: 3849.0, Score: 0.576494626, Count: 5.0
4125
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/2/2.csv  is csv_path and 
Processing tracking : 2
The two highest occurrences: 
First id: 4527.0, Score: 2.04974076, Count: 17.0
Second id: 4423.0, Score: 0.40846930400000003, Count: 3.0
4527
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\HOKKAIDO_PAPER\HOKKAIDO_BWU37/3/3.csv  is csv_path and 
BLACK
tracking  3  is black

In [ ]:
cv2.destroyAllWindows()

In [ ]:
summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 

In [ ]:
cap.release()
cv2.destroyAllWindows()


In [ ]:
dataset = "D:\\815_CowDataChecking\\20220704\\20220704_M_Cow" #don't add \\ at the end
#save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_track')  #open this when running single video
save_vid_name = dataset.split("\\")[-1] # open this when running multiple videos
print(save_vid_name)


In [ ]:
        

###### write final csv
for csv_index in range(len(MAX_prevId)):
    df = pd.DataFrame(MAX_prevId[csv_index] , columns = ['ID'])
    tracked_id = MAX_prevId[csv_index][0]
    try:
        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
        df["Original"] = org_ids
    except:
        df["Original"] = MAX_orgId[csv_index]
    now=str(datetime.now().date())

    save_csv_each_path = str(Path(save_dir / str(tracked_id) / f'{str(tracked_id)}.csv'))
    df.to_csv(save_csv_each_path, index= False)##asdfasdf
#All records     
data_to_add = {
    'ImageId': IMAGE_ID_LIST,
    'LocalId': LOCAL_ID_LIST,
    'xyxy1': MAX_xyxy1,
    'xyxy2': MAX_xyxy2,
    'xyxy3': MAX_xyxy3,
    'xyxy4': MAX_xyxy4
    # Add more columns as needed
}
add_lines_to_excel(csv_main_file_path, data_to_add)

IMAGE_STORED_LOCATION = []

cattle_ids = []
#################################\
manual_summarize_ids = []
manual_local_ids = []

manual_summarize_ids = []
manual_local_ids = []
#### write video after saving csv
final_cattle_count = 1



save_vid_name=  dataset.split("\\")[-1].replace('.mkv','')+'_classification'
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
classification_vid = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 13, (1920,1080))

manual_summarize_ids = get_final_cattle_id(save_dir,CATTLE_SAVED_COUNT)
#for loc in range(len(demo_img_save_path)):
#    print(demo_img_save_path[loc])
#    final_cattle_id = HOKKAIDO_WRITE_VIDEO(classification_vid,demo_img_save_path[loc])
#    if(final_cattle_id != -1):
#        manual_local_ids.append(final_cattle_count)
#        manual_summarize_ids.append(final_cattle_id)
#        final_cattle_count+=1 



print(save_vid_path)
#### write video after saving csv
manual_summarize_ids = get_final_cattle_id(str(save_dir),TOTAL_CATTLE_COUNT)

summarize_id_csv = pd.DataFrame(manual_summarize_ids, columns = ["Cow Id"])

summarize_id_csv.to_csv(str(save_dir)+'/summarize_id_'+now+'.csv', index= False) 
csv_name = 'main_csv.csv'
#region Write Video
try:
    writeVideo(classification_vid,str(save_dir),csv_name ,manual_summarize_ids,image_count)
except:
    print(" Done")

classification_vid.release()
cv2.destroyAllWindows()
#prevId_record = []
#MAX_prevId = []
#MAX_xyxy1 = [] 
#MAX_xyxy2 = [] 
#MAX_xyxy3 = [] 
#MAX_xyxy4 = [] 
#MAX_orgId = [] 

In [ ]:
import pandas as pd
csv_path = "runs\\segment\\honkawa_1103_112_RANK\\03_1_video\\identification35\\8\\8.csv"
testing = pd.read_csv(csv_path)
print((testing['RANK1'] == 10172).sum())
print(len(testing['RANK1']))